In [1]:
from pre_process_2 import get_final
import os
from PIL import Image
from pre_process_original import *
import cv2
from model import SegmentationModel


root = os.getcwd()
data = "Data"
cellname = "cell02_APPL1_GFP"
imgs = list(sorted(os.listdir(os.path.join(root, data,cellname))))

img_path = os.path.join(root,data,cellname, imgs[0])

img = get_final(img_path, False)

d = img.shape[0]
img_array = []
for i in range (0,d):
    m_new = img[i]
    img_array.append(np.array(m_new, dtype=np.float32))

k = np.amax(img_array)
img_array = np.array(img_array)
img_array = (img_array)/float(k)

img_array = np.expand_dims(img_array,axis=0)
print(img_array.shape)

model = SegmentationModel(checkpoint="checkpoint_train.pth.tar", mode="eval")

mask = model.inference(img_array)

print(mask.shape)



(1, 128, 128, 128)
torch.Size([128, 128, 128])


In [4]:

def get_binary_image2(img: Image):
 
        images = []
        for i in range (0,img.n_frames):
            res = img.seek(i)
            images.append(np.array(img,dtype=np.float32))
        images = np.array(images,dtype=np.float32)
        # 80,608,400
        #image.shape = 608,608,608
        d = images.shape[0]
        h = images.shape[1]
        w = images.shape[2]

        max_dim = max(max(h,w),d)
        print(max_dim)

        d_up = int(math.ceil(float((max_dim-d)/2)))
        d_down = int(math.floor(float((max_dim-d)/2)))
        print(d_up,d_down)

        w_up = int(math.ceil(float((max_dim-w)/2)))
        w_down = int(math.floor(float((max_dim-w)/2)))
        print(w_up,w_down)

        h_up = int(math.ceil(float((max_dim-h)/2)))
        h_down = int(math.floor(float((max_dim-h)/2)))
        print(h_up,h_down)

        images = np.pad(images,((d_up,d_down),(0,0),(0,0)),'wrap')
        print(images.shape)
        images = np.pad(images,((0,0),(0,0),(w_up,w_down)),'wrap')
        print(images.shape)
        images = np.pad(images,((0,0),(h_up,h_down),(0,0)),'wrap')
        
        print(images.shape)

        return (images,d,h,w,max_dim)

In [5]:


def get_final2(path_dir_1,mask):
    
    img = Image.open(path_dir_1)
    
    output = get_binary_image2(img)
    images = output[0]
    d = output[1]
    h = output[2]
    w = output[3]
    max_dim = output[4]

    print("output: ", output[1:])
    print((max_dim-d)/2)
    start = math.ceil(float((max_dim-d)/2))
    if (start!=0):
        start = start+1
    end = start+128
    print("start: ", start, "end: ", end)
    
    final = []

    for i in range (start,end):
        img = images[i]
        res = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
        print(i, img.shape, res.shape)
        final.append(np.array(res))

    out = np.array(final)
    print(out.shape)
 
    return out


img2 = get_final2(img_path, False)

assert np.array_equal(img, img2)

608
264 264
104 104
0 0
(608, 608, 400)
(608, 608, 608)
(608, 608, 608)
output:  (80, 608, 400, 608)
264.0
start:  265 end:  393
265 (608, 608) (128, 128)
266 (608, 608) (128, 128)
267 (608, 608) (128, 128)
268 (608, 608) (128, 128)
269 (608, 608) (128, 128)
270 (608, 608) (128, 128)
271 (608, 608) (128, 128)
272 (608, 608) (128, 128)
273 (608, 608) (128, 128)
274 (608, 608) (128, 128)
275 (608, 608) (128, 128)
276 (608, 608) (128, 128)
277 (608, 608) (128, 128)
278 (608, 608) (128, 128)
279 (608, 608) (128, 128)
280 (608, 608) (128, 128)
281 (608, 608) (128, 128)
282 (608, 608) (128, 128)
283 (608, 608) (128, 128)
284 (608, 608) (128, 128)
285 (608, 608) (128, 128)
286 (608, 608) (128, 128)
287 (608, 608) (128, 128)
288 (608, 608) (128, 128)
289 (608, 608) (128, 128)
290 (608, 608) (128, 128)
291 (608, 608) (128, 128)
292 (608, 608) (128, 128)
293 (608, 608) (128, 128)
294 (608, 608) (128, 128)
295 (608, 608) (128, 128)
296 (608, 608) (128, 128)
297 (608, 608) (128, 128)
298 (608, 608

In [10]:
import test

test.main()

Loading Checkpoint
Loaded 200 test images from /Users/patrickcleeve/Documents/university/bio/demarco/4D-Segmentation/Data/cell02_APPL1_GFP.
Loaded checkpoint: checkpoint_train.pth.tar on cpu device.


Image 0/200: Model Inference:   0%|          | 0/200 [00:01<?, ?it/s]

TYPE:  <class 'torch.Tensor'>


# Load Images


In [13]:
%load_ext autoreload
%autoreload 2


from pre_process_2 import get_final, get_binary_image
import numpy as np
import math

import numpy as np
import sys
from tifffile import imsave
from pre_process_original import *
import cv2
from PIL import Image

def get_final_old(path_dir_1, mask, truth: bool):

    if truth:
        img = Image.open(path_dir_1)
        output = get_binary_image(img)
        images = output[0]
        return np.array(images)

    else:
        img = Image.open(path_dir_1)
        output = get_binary_image(img)
        images = output[0]
        d = output[1]
        h = output[2]
        w = output[3]
        max_dim = output[4]

        start = math.ceil(float((max_dim - d) / 2))
        if start != 0:
            start = start + 1
        end = start + 128

        final = []

        for i in range(start, end):
            img = images[i]
            res = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
            final.append(np.array(res))

        out = np.array(final)
        # print(out.shape)

        return np.array(out)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:



fname = "test1.tif"

img1 = get_final(fname, False, downsample=False)
img2 = get_final(fname, False, downsample=True)
img11 = get_final_old(fname, False, truth=True)
img22 = get_final_old(fname, False, truth=False)


assert np.array_equal(img1, img11)
assert np.array_equal(img2, img22)